In [1]:
import pandas as pd
import json
import re
import sys
from time import time, sleep
import pymysql

In [2]:
from detect import hasChinese, countChinese

In [3]:
from toEnglish import toEnglish, addLog, getIRetry, resetIRetry, setAutoReset

In [ ]:
setAutoReset(False)

In [4]:
from sendemail import sendEmail

In [5]:
sys.path += [sys.path[0] + '/../py214 testPymysql']
from vmysql import addCursor, sql, select, describe, count

In [6]:
toEnglish('你好!')

'Hello there!'

In [7]:
def ourError (s, error = 'invalid sentence'):
    with open('error.txt', 'a', encoding = 'utf-8') as f:
        f.write(f'[ERROR] {now}, {bulk} : {error}: "{s}"\n')
addLog(ourError)

In [8]:
def ourLog (s, log = ''):
    with open('log.txt', 'a', encoding = 'utf-8') as f:
        f.write(f'[LOG] {now}, {bulk} : {log} : "{s}"\n')

In [9]:
db = pymysql.connect("localhost", "root", "linlei",port=13306,charset='utf8',db='beihangEng')
cursor = db.cursor()
addCursor(cursor)

In [10]:
now = 0
bulk = 100

In [11]:
sSelect = '''
select id, paper_title as title, abst as abstract, keywords
from beihang.paper_20201012
limit %d, %d
'''

In [12]:
sInsert = '''
insert into `paper` (`id`, `title`, `abstract`, `keywords`) values
%s
'''

In [13]:
def frmt (s):
    if getIRetry(): print()
    print (f'{now}, {bulk}', s, ' ' * 20, end='\r')
    resetIRetry()

In [14]:
def std (s):
    return repr(s) if s else 'null'

In [ ]:
def rmUnsee (s):
    return re.sub('(\s)+', ' ', s) if s else ''

In [15]:
def list2Eng (l, sep = '\n\n^^\n\n'):
    l = [rmUnsee(x).strip() for x in l]
    rst = toEnglish(sep.join(l)).split(sep)
    if len(l) != len(rst): ourError((l,rst), 'translate lines error')
    return rst

In [16]:
def solve (now, bulk):
    if now // bulk % 40 == 0: print()
    frmt('selecting data')
    start = time()
    data = sql(sSelect % (now, bulk))
    ourLog(f'{time() - start:.3f}s', 'select time')
    
    ids = data['id']

    frmt('translating titles')
    start = time()
    titles = list2Eng(data['title'].tolist())
    ourLog(f'{time() - start:.3f}s', 'translate titles time')
    
    frmt('translating abstracts')
    start = time()
    abstracts = list2Eng(data['abstract'].tolist())
    ourLog(f'{time() - start:.3f}s', 'translate abstracts time')
    
    frmt('translating keywordss')
    start = time()
    keywordss = list2Eng(data['keywords'].tolist())
    ourLog(f'{time() - start:.3f}s', 'translate keywordss time')
    
    sIns = sInsert % ','.join([
        f'({std(x)}, {std(y)}, {std(z)}, {std(w)})' 
        for x, y, z, w in zip(ids,titles,abstracts,keywordss)
    ])
    if hasChinese(sIns): ourError(f'{countChinese(sIns)}', 'chinese count')

    frmt('inserting')
    start = time()
    sql (sIns)
    db.commit()
    ourLog(f'{time() - start:.3f}s', 'insert time')

In [17]:
%%time
count('beihang.paper_20201012').values.item()

Wall time: 3min 38s


4858623

In [18]:
def convertSeconds (n):
    d = h = m = s = 0
    s = n % 60
    n = round(n // 60)
    m = n % 60
    if not n: return f'{s:.3f} s'
    n //= 60
    h = n % 24
    if not n: return f'{m} min {s:.3f} s'
    d = n // 24
    if not d: return f'{h}h {m}min {s:.3f}s'
    return f'{d} d {h} h {m} min {s:.3f} s'

In [22]:
bulk = 200

In [20]:
# now = 512800

In [25]:
now, bulk

(512800, 200)

In [24]:
%%time
start = time()
nRows = count('beihang.paper_20201012').values.item()
while now < nRows:
    solve (now, bulk)
    now += bulk
sendEmail(f'cost time {convertSeconds(time() - start)}', 'translating papers accomplished')


1400, 200 translating abstracts                     
network error, retrying   18
2400, 200 translating abstracts                     
network error, retrying   13
2600, 200 translating titles                        
network error, retrying   2
2600, 200 translating abstracts                     
network error, retrying   3
3800, 200 translating titles                        
network error, retrying   4
3800, 200 translating abstracts                     
network error, retrying   16
5000, 200 translating abstracts                     
network error, retrying   18
5600, 200 translating abstracts                     
network error, retrying   18
6600, 200 translating abstracts                     
network error, retrying   19
7800, 200 translating abstracts                     
network error, retrying   4
7800, 200 inserting                                 
8000, 200 translating abstracts                     
network error, retrying   5
8200, 200 translating titles                     

44400, 200 translating abstracts                      translating abstracts                     
network error, retrying   14
44600, 200 translating abstracts                     
network error, retrying   2
46400, 200 translating abstracts                      inserting                     
network error, retrying   1
46600, 200 translating abstracts                     
network error, retrying   6
46800, 200 translating titles                        
network error, retrying   1
46800, 200 translating abstracts                     
network error, retrying   7
47800, 200 inserting                                 
48000, 200 translating abstracts                     
network error, retrying   2
48800, 200 translating abstracts                     
network error, retrying   5
48800, 200 translating keywordss                     
network error, retrying   1
49000, 200 translating abstracts                     
network error, retrying   3
49200, 200 translating abstracts                   

89000, 200 translating titles                     
network error, retrying   1
89000, 200 translating abstracts                     
network error, retrying   10
90000, 200 translating abstracts                      selecting data                     
network error, retrying   3
90200, 200 translating abstracts                     
network error, retrying   4
90400, 200 translating titles                        
network error, retrying   1
90400, 200 translating abstracts                     
network error, retrying   2
90600, 200 translating abstracts                     
network error, retrying   5
90800, 200 translating abstracts                     
network error, retrying   1
91800, 200 translating keywordss                      selecting data                     
network error, retrying   3
92000, 200 translating abstracts                     
network error, retrying   8
92000, 200 translating keywordss                     
network error, retrying   1
92200, 200 translating title

125200, 200 translating titles                     g data                     
network error, retrying   2
125200, 200 translating abstracts                     
network error, retrying   13
126400, 200 translating abstracts                      selecting data                      selecting data                     
network error, retrying   13
126600, 200 translating abstracts                      selecting data                     
network error, retrying   5
127800, 200 translating abstracts                     
network error, retrying   11
127800, 200 translating keywordss                     
network error, retrying   2
127800, 200 inserting                     
128000, 200 translating titles                     
network error, retrying   1
128000, 200 translating abstracts                     
network error, retrying   4
129000, 200 translating keywordss                      selecting data                     
network error, retrying   5
129200, 200 translating titles            

172200, 200 translating abstracts                      selecting data                      selecting data                     
network error, retrying   9
172200, 200 translating keywordss                     
network error, retrying   1
172400, 200 translating abstracts                     
network error, retrying   9
173600, 200 translating abstracts                      selecting data                     
network error, retrying   18
174800, 200 translating abstracts                     
network error, retrying   15
175000, 200 translating abstracts                      selecting data                     
network error, retrying   3
175800, 200 inserting                                 
176200, 200 translating abstracts                     
network error, retrying   18
177600, 200 translating abstracts                      selecting data                     
network error, retrying   18
179000, 200 translating abstracts                      selecting data                      select

...

376800, 200 translating titles                        
network error, retrying   1
376800, 200 translating abstracts                     
network error, retrying   2
377800, 200 translating abstracts                     
network error, retrying   17
378800, 200 translating abstracts                     
network error, retrying   11
379000, 200 translating abstracts                     
network error, retrying   6
379400, 200 translating abstracts                     
network error, retrying   1
380000, 200 translating keywordss                      selecting data                     
network error, retrying   1
380200, 200 translating abstracts                     
network error, retrying   4
380400, 200 translating abstracts                     
network error, retrying   2
380600, 200 translating abstracts                     
network error, retrying   2
380800, 200 translating abstracts                     
network error, retrying   6
381800, 200 translating abstracts                

network error, retrying   4
409800, 200 translating keywordss                     
network error, retrying   1
411000, 200 translating abstracts                      selecting data                     
network error, retrying   10
411200, 200 translating abstracts                     
network error, retrying   6
412200, 200 translating abstracts                      selecting data                     
network error, retrying   3
412800, 200 translating abstracts                     
network error, retrying   1
413200, 200 translating abstracts                     
network error, retrying   6
413400, 200 translating titles                        
network error, retrying   4
413400, 200 translating abstracts                     
network error, retrying   7
414400, 200 translating abstracts                     
network error, retrying   1
414600, 200 translating titles                         selecting data                     
network error, retrying   1
414600, 200 translating abstracts

445800, 200 translating abstracts                      selecting data                      selecting data                      selecting data                     
network error, retrying   3
447000, 200 translating abstracts                      selecting data                     
network error, retrying   6
447200, 200 translating abstracts                     
network error, retrying   3
447200, 200 translating keywordss                     
network error, retrying   1
447400, 200 translating abstracts                     
network error, retrying   5
447800, 200 inserting                                 
448400, 200 translating abstracts                     
network error, retrying   2
448400, 200 translating keywordss                     
network error, retrying   1
449400, 200 translating keywordss                     
network error, retrying   3
449600, 200 translating abstracts                     
network error, retrying   7
449800, 200 translating abstracts                     

483600, 200 translating abstracts                     
network error, retrying   1
483800, 200 translating abstracts                     
network error, retrying   4
484000, 200 translating abstracts                     
network error, retrying   2
484800, 200 translating abstracts                     
network error, retrying   4
485000, 200 translating abstracts                     
network error, retrying   3
485000, 200 translating keywordss                     
network error, retrying   2
485200, 200 translating abstracts                     
network error, retrying   4
486200, 200 translating abstracts                     
network error, retrying   6
486400, 200 translating abstracts                     
network error, retrying   10
487400, 200 translating abstracts                     
network error, retrying   7
487600, 200 translating titles                        
network error, retrying   1
487600, 200 translating abstracts                     
network error, retrying   8
487

OperationalError: (2013, 'Lost connection to MySQL server during query ([WinError 10053] 你的主机中的软件中止了一个已建立的连接。)')

In [38]:
db.close()